In [12]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn


In [22]:
FEATURE_COLS = [
    "HR", "O2Sat", "Temp", "SBP", "MAP", "Resp", "pH",
    "BUN", "WBC", "Platelets", "Magnesium", "Potassium"
]
LABEL_COL = "SepsisLabel"
META_COLS = ["PatientID", "Hour"]

def build_patient_sequences_simple(csv_path):
    df = pd.read_csv(csv_path)
    df = df.sort_values(["PatientID", "Hour"]).reset_index(drop=True)

    # Keep only what we care about
    keep_cols = META_COLS + FEATURE_COLS + [LABEL_COL]
    df = df[keep_cols]

    patients = []

    # Precompute global means for fallback impute at the end
    global_means = df[FEATURE_COLS].mean(skipna=True)

    for pid, g in df.groupby("PatientID"):
        g = g.sort_values("Hour")

        # Patient label: septic if ever SepsisLabel==1
        y_patient = int(g[LABEL_COL].fillna(0).max())

        # Grab features only
        X = g[FEATURE_COLS].copy()

        # 1. forward fill within patient
        X = X.ffill()

        # 2. fill any still-missing with global means
        X = X.fillna(global_means)

        # convert to tensor [T, F]
        X_tensor = torch.tensor(X.to_numpy(dtype=np.float32))

        patients.append({
            "patient_id": pid,
            "series": X_tensor,                        # [T, F]
            "label": torch.tensor(y_patient).float(), # scalar
            "length": X_tensor.shape[0],
        })

    in_features = len(FEATURE_COLS)
    return patients, in_features

In [23]:
from torch.utils.data import Dataset, DataLoader

class TCNDataset(Dataset):
    def __init__(self, patients_list):
        self.patients = patients_list

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        p = self.patients[idx]
        return {
            "series": p["series"],        # [T, F]
            "label": p["label"],          # scalar
            "length": p["length"],        # int
            "patient_id": p["patient_id"]
        }

In [24]:
def collate_tcn(batch):
    lengths = [b["length"] for b in batch]
    max_len = max(lengths)
    feat_dim = batch[0]["series"].shape[1]
    B = len(batch)

    feats = torch.zeros(B, max_len, feat_dim, dtype=torch.float32)
    time_mask = torch.zeros(B, max_len, dtype=torch.float32)
    labels = torch.zeros(B, dtype=torch.float32)
    last_idx = torch.zeros(B, dtype=torch.long)

    patient_ids = []

    for i, b in enumerate(batch):
        T = b["length"]
        feats[i, :T, :] = b["series"]
        time_mask[i, :T] = 1.0
        labels[i] = b["label"]
        last_idx[i] = T - 1
        patient_ids.append(b["patient_id"])

    # TCN expects [B, C, T]
    feats = feats.permute(0, 2, 1)

    return {
        "x": feats,            # [B, feat_dim, max_len]
        "mask": time_mask,     # [B, max_len] (not strictly needed for patient-level loss)
        "y": labels,           # [B]
        "last_idx": last_idx,  # [B]
        "patient_id": patient_ids
    }

In [25]:
class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, dropout=0.1):
        super().__init__()
        pad = (kernel_size - 1) * dilation

        self.conv1 = nn.Conv1d(in_ch, out_ch,
                               kernel_size=kernel_size,
                               padding=pad,
                               dilation=dilation)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_ch, out_ch,
                               kernel_size=kernel_size,
                               padding=pad,
                               dilation=dilation)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(dropout)

        self.residual = (
            nn.Conv1d(in_ch, out_ch, kernel_size=1)
            if in_ch != out_ch else nn.Identity()
        )

        self._init_weights()

    def _init_weights(self):
        for m in [self.conv1, self.conv2]:
            nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        if isinstance(self.residual, nn.Conv1d):
            nn.init.kaiming_normal_(self.residual.weight)
            if self.residual.bias is not None:
                nn.init.zeros_(self.residual.bias)

    def forward(self, x):
        # x: [B, C, T]
        T = x.size(-1)

        y = self.conv1(x)[:, :, :T]
        y = self.relu1(y)
        y = self.drop1(y)

        y = self.conv2(y)[:, :, :T]
        y = self.relu2(y)
        y = self.drop2(y)

        res = self.residual(x)[:, :, :T]

        return y + res  # residual

class TCNModel(nn.Module):
    def __init__(self, in_ch, hidden_ch=64, num_levels=4, kernel_size=3):
        super().__init__()
        layers = []
        ch_in = in_ch
        for i in range(num_levels):
            dilation = 2 ** i  # 1,2,4,8,...
            ch_out = hidden_ch
            layers.append(
                TemporalBlock(
                    in_ch=ch_in,
                    out_ch=ch_out,
                    kernel_size=kernel_size,
                    dilation=dilation,
                    dropout=0.1,
                )
            )
            ch_in = ch_out

        self.tcn = nn.Sequential(*layers)
        self.head = nn.Conv1d(ch_in, 1, kernel_size=1)  # per-timestep logit

    def forward(self, x):
        # x: [B, in_ch, T]
        h = self.tcn(x)        # [B, hidden_ch, T]
        logits = self.head(h)  # [B, 1, T]
        logits = logits.squeeze(1)  # [B, T]
        return logits

In [26]:
def train_step(model, batch, optimizer, device):
    model.train()
    x = batch["x"].to(device)             # [B, C, T]
    y = batch["y"].to(device)             # [B]
    last_idx = batch["last_idx"].to(device)

    logits_time = model(x)                # [B, T]
    B = logits_time.shape[0]
    logits_last = logits_time[torch.arange(B, device=device), last_idx]

    loss_fn = nn.BCEWithLogitsLoss()
    loss = loss_fn(logits_last, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        probs = torch.sigmoid(logits_last)
        preds = (probs > 0.5).float()
        acc = (preds == y).float().mean()

    return loss.item(), acc.item()

In [ ]:
#File path for training data. 

csv_path = "subset_train.csv"
patients, in_ch = build_patient_sequences_simple(csv_path)

dataset = TCNDataset(patients)
loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_tcn,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TCNModel(in_ch=in_ch, hidden_ch=64, num_levels=4, kernel_size=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    for batch in loader:
        loss, acc = train_step(model, batch, optimizer, device)
    print(f"epoch {epoch} | loss={loss:.4f} | acc={acc:.4f}")

epoch 0 | loss=8.7165 | acc=0.8750
epoch 1 | loss=0.5894 | acc=0.8750
epoch 2 | loss=0.0000 | acc=1.0000
epoch 3 | loss=0.0000 | acc=1.0000
epoch 4 | loss=4.6714 | acc=0.8750


In [32]:
#train_csv = "train.csv"
val_csv   = "subset_train.csv"

#train_patients, in_ch = build_patient_sequences_simple(train_csv)
val_patients, _       = build_patient_sequences_simple(val_csv)

#train_dataset = TCNDataset(train_patients)
val_dataset   = TCNDataset(val_patients)

#train_loader = DataLoader(
#    train_dataset,
#    batch_size=16,
#    shuffle=True,
#    collate_fn=collate_tcn,
#)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,        # important: don't shuffle eval
    collate_fn=collate_tcn,
)

In [33]:
@torch.no_grad()
def eval_epoch(model, loader, device):
    model.eval()
    all_losses = []
    all_accs = []

    all_probs = []
    all_targets = []

    loss_fn = torch.nn.BCEWithLogitsLoss()

    for batch in loader:
        x = batch["x"].to(device)             # [B, C, T]
        y = batch["y"].to(device)             # [B]
        last_idx = batch["last_idx"].to(device)

        logits_time = model(x)                # [B, T]
        B = logits_time.shape[0]
        logits_last = logits_time[torch.arange(B, device=device), last_idx]  # [B]

        loss = loss_fn(logits_last, y)

        probs = torch.sigmoid(logits_last)    # [B]
        preds = (probs > 0.5).float()
        acc = (preds == y).float().mean()

        all_losses.append(loss.item())
        all_accs.append(acc.item())

        all_probs.append(probs.cpu())
        all_targets.append(y.cpu())

    # concat for possible AUROC / etc.
    all_probs = torch.cat(all_probs)      # shape [N_val_patients]
    all_targets = torch.cat(all_targets)  # shape [N_val_patients]

    avg_loss = float(torch.tensor(all_losses).mean())
    avg_acc  = float(torch.tensor(all_accs).mean())

    return {
        "val_loss": avg_loss,
        "val_acc": avg_acc,
        "probs": all_probs,
        "targets": all_targets,
    }

In [34]:
model.eval()                # switch to inference mode
metrics = eval_epoch(model, val_loader, device)
print(metrics)

{'val_loss': 0.7443217039108276, 'val_acc': 0.96875, 'probs': tensor([1.8070e-18, 1.0000e+00, 7.7453e-18, 2.7479e-05, 4.3148e-01, 4.3466e-22,
        8.0270e-07, 9.6127e-06, 1.0522e-11, 1.0000e+00, 9.0951e-13, 5.6985e-09,
        1.6963e-16, 3.0734e-10, 2.2778e-19, 3.0689e-37, 8.7208e-01, 0.0000e+00,
        2.3087e-11, 3.3978e-20, 0.0000e+00, 5.6141e-12, 1.2921e-28, 1.0856e-25,
        0.0000e+00, 0.0000e+00, 2.9066e-28, 1.1929e-23, 1.1174e-11, 2.3394e-09,
        6.0702e-27, 1.0826e-09, 4.1535e-09, 3.4999e-09, 1.6795e-11, 5.2527e-08,
        2.1569e-25, 1.3614e-22, 1.0000e+00, 2.9687e-19, 0.0000e+00, 3.0680e-21,
        3.1024e-10, 1.6808e-18, 6.3880e-34, 1.9359e-22, 0.0000e+00, 1.2504e-38,
        7.8938e-31, 5.6591e-11, 2.2481e-25, 1.9720e-16, 6.5417e-17, 0.0000e+00,
        2.7177e-14, 4.1250e-29, 3.0286e-30, 3.0636e-38, 2.6874e-29, 7.4694e-24,
        0.0000e+00, 5.2262e-34, 2.8284e-11, 5.0453e-26, 1.9816e-20, 3.5343e-21,
        2.7986e-37, 0.0000e+00, 9.9981e-01, 1.4325e-25, 4.

In [35]:
from sklearn.metrics import roc_auc_score, average_precision_score

val_auc = roc_auc_score(metrics["targets"].numpy(),
                        metrics["probs"].numpy())
val_auprc = average_precision_score(metrics["targets"].numpy(),
                                    metrics["probs"].numpy())
print("val AUROC:", val_auc, "val AUPRC:", val_auprc)

val AUROC: 0.9069069069069069 val AUPRC: 0.7461920795254129
